In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [2]:
train_df = pd.read_csv('input/train.csv')
#train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
train_df.head()

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [6]:
# local cv: 0.9000721582954704
# lb: 0.899 (best)
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.04019176517639987,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

In [7]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=2000000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 800)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits
    score += model.best_score['valid_1']['auc'] / n_splits

print(score)

Fold 0 started at Wed Mar  6 14:44:29 2019
Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.838698	valid_1's auc: 0.831053
[600]	training's auc: 0.868296	valid_1's auc: 0.858332
[900]	training's auc: 0.883493	valid_1's auc: 0.872027
[1200]	training's auc: 0.892398	valid_1's auc: 0.879677
[1500]	training's auc: 0.898439	valid_1's auc: 0.884485
[1800]	training's auc: 0.902729	valid_1's auc: 0.888121
[2100]	training's auc: 0.906119	valid_1's auc: 0.890674
[2400]	training's auc: 0.908829	valid_1's auc: 0.892735
[2700]	training's auc: 0.910894	valid_1's auc: 0.894167
[3000]	training's auc: 0.912507	valid_1's auc: 0.895564
[3300]	training's auc: 0.913944	valid_1's auc: 0.896499
[3600]	training's auc: 0.91515	valid_1's auc: 0.897274
[3900]	training's auc: 0.916172	valid_1's auc: 0.897941
[4200]	training's auc: 0.917139	valid_1's auc: 0.898307
[4500]	training's auc: 0.91793	valid_1's auc: 0.898651
[4800]	training's auc: 0.918697	valid_1's auc: 0.898922
[51

In [8]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)